In [106]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [107]:
# Initiate pyimagej (at fiji mode)
import imagej
ij = imagej.init('sc.fiji:fiji')

In [117]:
def load_cubes_im3(data_path, metadata_path=None, sheet_name=0):
    data_path = Path(data_path)

    # --- only *.im3 files ----------------------------------------------------
    cube_names = sorted(
        f.name for f in data_path.iterdir()
        if f.is_file() and f.suffix.lower() == ".im3"
    )

    cubes = {}
    for name in cube_names:
        print(f"Loading {name} …")
        img  = ij.io().open(str(data_path / name))
        cube = np.array(ij.py.from_java(img), dtype=float)

        excitation = float(name[:-4])
        em_start = 420.0 if excitation <= 400.0 else excitation + 20.0
        em_end   = 720
        step = 10
        cubes[name] = {
            "ex": float(name[:-4]),   # e.g. 300 from “300.im3”
            "em_start": em_start,
            "em_end": em_end,
            "step": step,
            "num_rows": cube.shape[0],
            "num_cols": cube.shape[1],
            "num_bands": cube.shape[2],
            "expos_val": None,
            "notes": None,
            "data": cube,
            "em_arr": [k for k in range(int(em_start), int(em_end), int(step))]
        }

    # --- read Excel metadata (Excitation | Exposure) ------------------------
    if metadata_path:
        df = (
            pd.read_excel(metadata_path, sheet_name=sheet_name)
              .rename(columns=str.strip)          # trim any stray spaces
        )

        # build a dict: {excitation_value: exposure_value}
        exposure_lookup = dict(
            zip(df["Excitation"].astype(float), df["Exposure"])
        )

        # populate expos_val for every cube that has a match
        for cube_key, cube_dict in cubes.items():
            ex_val = cube_dict["ex"]
            if ex_val in exposure_lookup:
                cube_dict["expos_val"] = float(exposure_lookup[ex_val])
            else:
                print(f"  ⚠ no exposure value in metadata for Ex={ex_val}")

    return cubes

In [118]:
cues = load_cubes_im3('../Data/Kiwi', metadata_path='../Data/Kiwi/metadata.xlsx')

Loading 300.im3 …
Loading 310.im3 …
Loading 320.im3 …
Loading 330.im3 …
Loading 340.im3 …
Loading 350.im3 …
Loading 360.im3 …
Loading 370.im3 …
Loading 380.im3 …
Loading 390.im3 …
Loading 400.im3 …
Loading 410.im3 …
Loading 420.im3 …
Loading 430.im3 …
Loading 440.im3 …
Loading 450.im3 …
Loading 460.im3 …
Loading 470.im3 …
Loading 480.im3 …
Loading 490.im3 …
Loading 500.im3 …


In [128]:
cues['300.im3']['data'].shape

(1024, 1392, 31)